In [0]:
from pyspark.sql.utils import AnalysisException
import traceback

# -----------------------------------------
# Function to write Auto Loader streaming data to Delta table
# -----------------------------------------
def streaming_write_to_table(df, table_name, mode):
    try:
        print(f"{JOB_NAME} - Starting streaming write to Delta table: {table_name}")
        logger.info(f"{JOB_NAME} - Starting streaming write to Delta table: {table_name}")

        if df.isStreaming:
            checkpoint_path = f"{SOURCE_PATH}/_checkpoints/{JOB_NAME}"

            query = (
                df.writeStream
                  .format("delta")
                  .outputMode("append")  # streaming doesn't support overwrite
                  .option("checkpointLocation", checkpoint_path)
                  .trigger(once=True)  # batch-style incremental
                  .toTable(table_name)
            )

            logger.info(f"{JOB_NAME} - writeStream triggered successfully")
            query.awaitTermination()
            logger.info(f"{JOB_NAME} - writeStream completed for table: {table_name}")

        else:
            # Optional fallback: non-streaming support
            row_count = df.count()
            if row_count == 0:
                logger.warning(f"{JOB_NAME} - No data to write to table: {table_name}")
                return

            df.write.format("delta") \
                .mode(mode) \
                .saveAsTable(table_name)

            logger.info(f"{JOB_NAME} - Write successful to table: {table_name}")
            logger.info(f"{JOB_NAME} - Rows written: {row_count:,}")

    except AnalysisException as ae:
        logger.error(f"{JOB_NAME} - Table write failed due to analysis error")
        logger.error(f"{JOB_NAME} - {str(ae)}")
        logger.error(traceback.format_exc())
        raise RuntimeError(f"{JOB_NAME} - Analysis failure writing to {table_name}") from ae

    except Exception as e:
        logger.error(f"{JOB_NAME} - General failure during write to table: {table_name}")
        logger.error(f"{JOB_NAME} - Exception: {str(e)}")
        logger.error(traceback.format_exc())
        raise RuntimeError(f"{JOB_NAME} - Write failed to {table_name}") from e